In [1]:
!pip install tqdm

!apt-get update
!apt-get install -y wget

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,321 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,664 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jamm

In [2]:
!pip install selenium
!apt-get install -y hromium-browser
!apt-get install -y chromium-chromedriver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 19.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package hromium-browser
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 snapd squashfs-tools systemd-hwe-hwdb udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver libfuse3-3 liblzo2-2 snapd squashfs-tools
  systemd-hwe-hwdb udev
0 upgraded, 9 newly installed, 0 to remove and 32 not upgraded.
Need to get 30.1 MB of archives.
After this operation, 123 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-upd

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import urllib.request
import os
import time
from tqdm import tqdm
import concurrent.futures
import json
from PIL import Image



# WebURL Crawl

In [20]:
# @title
class UrlScraper:
  # Constructor
  def __init__(self, url_template, max_images=50, max_workers=4):
    self.url_template = url_template #Link crawl
    self.max_images = max_images #Max images
    self.max_workers = max_workers #Thread
    self.setup_environment() # call for set up environment


  def setup_environment(self):
    os.environ['PATH'] += ':/usr/lib/chromium-browser/'
    os.environ['PATH'] += ':/usr/lib/chromium-browser/chromedriver/'

  def get_url_images(self, term):
    """
    Crawl the urls of images by term

    Parameters:
    term (str): The name of animal, plant, scenery, furniture

    Returns:
    urls (list): List of urls of images
    """
    # Initialize Chrome driver
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)

    url = self.url_template.format(search_term=term)
    driver.get(url)

    # Start crawl urls of image like brute force - the same mechanism with this but add
    # some feature
    urls = []
    more_content_available = True

    pbar = tqdm(total=self.max_images, desc=f"Fetching images for {term}") # Set up for visualize progress

    while len(urls) < self.max_images and more_content_available:
        soup = BeautifulSoup(driver.page_source, "html.parser")
        img_tags = soup.find_all("img")

        for img in img_tags:
            if len(urls) >= self.max_images:
                break

            if 'src' in img.attrs:
                href = img.attrs['src']
                img_path = urljoin(url, href)
                img_path = img_path.replace("_m.jpg", "_b.jpg").replace("_n.jpg", "_b.jpg").replace("_w.jpg", "_b.jpg")

                if img_path == "https://combo.staticflickr.com/ap/build/images/getty/IStock_corporate_logo.svg":
                    continue

                urls.append(img_path)
                pbar.update(1)

        try:
            load_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//button[@id="yui_3_16_0_1_1721642285931_28620"]'))
            )
            load_more_button.click()

        except:
            time.sleep(2)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)

            new_soup = BeautifulSoup(driver.page_source, "html.parser")
            new_img_tags = new_soup.find_all("img", loading_="lazy")

            if len(new_img_tags) == len(img_tags):
                more_content_available = False

            img_tags = new_img_tags
    pbar.close()
    driver.quit()
    return urls


  def scrape_urls(self, categories):
    """
    Call get_url_images method to get all urls of any object in categories\

    Parameter:
    categories (dictionary): the dict of all object we need to collect image with format


    Returns:
    all_urls (dictionary): Dictionary of urls of images
    """
    all_urls = {category: {} for category in categories}

    # Handle multi-threading for efficient installation
    with concurrent.futures.ThreadPoolExecutor(max_workers=self.max_workers) as executor:
        future_to_term = {executor.submit(self.get_url_images, term): (category, term)
                          for category, terms in categories.items() for term in terms}

        for future in tqdm(concurrent.futures.as_completed(future_to_term), total=len(
                future_to_term), desc="Overall Progress"):
            category, term = future_to_term[future]
            try:
                urls = future.result()
                all_urls[category][term] = urls
                print(f"\nNumber of images retrieved for {term}: {len(urls)}")
            except Exception as exc:
                print(f"\n{term} generated an exception: {exc}")

    return all_urls

  def scrape_urls(self, categories):
    """
    Call get_url_images method to get all urls of any object in categories\

    Parameter:
    categories (dictionary): the dict of all object we need to collect image with format


    Returns:
    all_urls (dictionary): Dictionary of urls of images
    """
    all_urls = {category: {} for category in categories}

    # Handle multi-threading for efficient installation
    with concurrent.futures.ThreadPoolExecutor(max_workers=self.max_workers) as executor:
        future_to_term = {executor.submit(self.get_url_images, term): (category, term)
                          for category, terms in categories.items() for term in terms}

        for future in tqdm(concurrent.futures.as_completed(future_to_term), total=len(
                future_to_term), desc="Overall Progress"):
            category, term = future_to_term[future]
            try:
                urls = future.result()
                all_urls[category][term] = urls
                print(f"\nNumber of images retrieved for {term}: {len(urls)}")
            except Exception as exc:
                print(f"\n{term} generated an exception: {exc}")

    return all_urls


  def save_to_file(self, data, filename):
    """
    Save the data to a JSON file.
    Parameters:
    data (dict): The data to be saved.
    filename (str): The name of the JSON file.
    Returns:
    None
    """
    with open(filename, 'w') as file:
      json.dump(data, file, indent=4)
      print(f"Data saved to {filename}")

In [21]:
categories = {
    "animal": ["Monkey", "Elephant", "cows", "Cat", "Dog", "bear", "fox", "Civet", "Pangolins",
                "Rabbit", "Bats", "Whale", "Cock", "Owl", "flamingo", "Lizard", "Turtle", "Snake",
                "Frog", "Fish", "shrimp", "Crab", "Snail", "Coral", "Jellyfish", "Butterfly", "Flies",
                "Mosquito", "Ants", "Cockroaches", "Spider", "scorpion", "tiger", "bird", "horse",
                "pig", "Alligator", "Alpaca", "Anteater", "donkey", "Bee", "Buffalo", "Camel",
               "Caterpillar", "Cheetah", "Chicken", "Dragonfly", "Duck", "panda", "Giraffe"],
    "plant": ["Bamboo", "Apple", "Apricot", "Banana", "Bean", "Wildflower", "Flower",
                "Mushroom", "Weed", "Fern", "Reed", "Shrub", "Moss", "Grass", "Palmtree", "Corn",
                "Tulip", "Rose", "Clove", "Dogwood", "Durian", "Ferns", "Fig", "Flax", "Frangipani",
                "Lantana", "Hibiscus", "Bougainvillea", "Pea", "OrchidTree", "RangoonCreeper",
                "Jackfruit", "Cottonplant", "Corneliantree", "Coffeeplant", "Coconut", "wheat",
                "watermelon", "radish", "carrot"],
    "furniture": ["bed", "cabinet", "chair", "chests", "clock", "desks", "table", "Piano",
                "Bookcase", "Umbrella", "Clothes", "cart", "sofa", "ball", "spoon", "Bowl", "fridge",
                "pan", "book"],
    "scenery": ["Cliff", "Bay", "Coast", "Mountains", "Forests", "Waterbodies", "Lake",
                "desert", "farmland", "river", "hedges", "plain", "sky", "cave", "cloud", "flowergarden",
                "glacier", "grassland", "horizon", "lighthouse", "plateau", "savannah", "valley",
                "volcano", "waterfall"]
}

urltopic = {"flickr": "https://www.flickr.com/search/?text={search_term}"}
scraper = UrlScraper(url_template=urltopic["flickr"], max_images=20, max_workers=5)

image_urls = scraper.scrape_urls(categories)
scraper.save_to_file(image_urls, 'image_urls.json')

Fetching images for cows:   0%|          | 0/20 [00:00<?, ?it/s]

Fetching images for Monkey:   0%|          | 0/20 [00:00<?, ?it/s]


Fetching images for Cat:   0%|          | 0/20 [00:00<?, ?it/s]



Fetching images for Dog:   0%|          | 0/20 [00:00<?, ?it/s]




Fetching images for cows:   5%|▌         | 1/20 [00:03<01:07,  3.56s/it]



Fetching images for Dog:   5%|▌         | 1/20 [00:01<00:26,  1.39s/it]

Fetching images for Monkey:   5%|▌         | 1/20 [00:02<00:54,  2.87s/it]


Fetching images for Cat:   5%|▌         | 1/20 [00:02<00:53,  2.82s/it]




Overall Progress:   1%|          | 1/134 [00:52<1:56:53, 52.73s/it]


Number of images retrieved for Dog: 20


Fetching images for Elephant: 100%|██████████| 20/20 [00:19<00:00,  1.00it/s]


Number of images retrieved for Monkey: 20



Overall Progress:   2%|▏         | 3/134 [00:54<27:01, 12.38s/it]

...


Number of images retrieved for savannah: 20






Overall Progress:  99%|█████████▊| 132/134 [22:56<00:20, 10.27s/it]


Number of images retrieved for valley: 20

Number of images retrieved for volcano: 20


Overall Progress: 100%|██████████| 134/134 [23:04<00:00, 10.33s/it]



Number of images retrieved for waterfall: 20
Data saved to image_urls.json


## Collect Data

In [22]:
class ImageDownloader:
    def __init__(self, json_file, download_dir='Dataset', max_workers=4, delay=1):
        self.json_file = json_file  # file containing URLs of images in JSON format
        self.download_dir = download_dir  # Folder name for storing images
        self.max_workers = max_workers  # Number of threads
        self.delay = delay  # Polite delay: when we send request too much to the server for
                            # downloading images without polite delay, it will crash or prevent your IP
        self.filename = set()  # To store filename directories
        self.setup_directory()  # Set up the folder structure

    def setup_directory(self):
        if not os.path.exists(self.download_dir):
            os.makedirs(self.download_dir)

    def read_json(self):
        """
        Read the JSON file and return the data.

        Returns:
            data (dict): The data read from the JSON file.
        """
        with open(self.json_file, 'r') as file:
            data = json.load(file)
        return data

    def is_valid_url(self, url):
        """
        Check if the URL is valid.

        Parameters:
            url (str): The URL to be checked.

        Returns:
            bool: True if the URL is valid, False otherwise.
        """
        try:
            with urllib.request.urlopen(url) as response:
                if response.status == 200 and 'image' in response.info().get_content_type():
                    return True
        except Exception:
            return False

    def download_image(self, url, category, term, pbar):
        """
        Download the image from the given URL.

        Parameters:
            url (str): The URL of the image to be downloaded.
            category (str): The category of the image.
            term (str): The term or keyword associated with the image.
            pbar (tqdm): The progress bar object.

        Returns:
            str: A message indicating the status of the download.
        """
        if not self.is_valid_url(url):
            pbar.update(1)
            return f"Invalid URL: {url}"

        category_dir = os.path.join(self.download_dir, category)
        if not os.path.exists(category_dir):
            os.makedirs(category_dir)

        term_dir = os.path.join(category_dir, term)
        if not os.path.exists(term_dir):
            os.makedirs(term_dir)

        filename = os.path.join(term_dir, os.path.basename(urlparse(url).path))

        self.filename.add(filename)  # Record the filename directory

        try:
            urllib.request.urlretrieve(url, filename)
            pbar.update(1)
            return f"Downloaded: {url}"

        except Exception as e:
            pbar.update(1)
            return f"Failed to download {url}: {str(e)}"


    def download_images(self):
        """
        Download images from the JSON file.

        Returns:
            None
        """
        data = self.read_json()
        download_tasks = []

        total_images = sum(len(urls) for terms in data.values() for urls in terms.values())
        with tqdm(total=total_images, desc="Downloading images") as pbar:
            with concurrent.futures.ThreadPoolExecutor(max_workers=self.max_workers) as executor:
                for category, terms in data.items():
                    for term, urls in terms.items():
                        for url in urls:
                            download_tasks.append(executor.submit(self.download_image, url, category, term, pbar))
                            time.sleep(self.delay)  # Polite delay

                for future in concurrent.futures.as_completed(download_tasks):
                    print(future.result())

        self.export_filename()

    def export_filename(self):
        """
        Export the filename directories to a text file.

        Returns:
            None
        """
        with open('filename.txt', 'w') as file:
            for filename in sorted(self.filename):
                file.write(f"{filename}\n")

In [23]:
downloader = ImageDownloader(json_file='image_urls.json', download_dir='Dataset', max_workers=4, delay=1)

downloader.download_images()
downloader.export_filename()

Downloaded: https://live.staticflickr.com/3498/3810185145_6538499fc4_b.jpg
Downloaded: https://live.staticflickr.com/2506/4169729385_e6e914e173_b.jpg
Downloaded: https://live.staticflickr.com/8308/7913975932_7743ede2a0_b.jpg
Downloaded: https://live.staticflickr.com/911/42092241251_245d13f5dd_b.jpg
Downloaded: https://live.staticflickr.com/2894/13211662814_fb156ec820_b.jpg
Downloaded: https://live.staticflickr.com/2185/1553488364_48d01c9862_b.jpg
Downloaded: https://live.staticflickr.com/5582/14594437783_765106a278_b.jpg
Downloaded: https://live.staticflickr.com/7333/27401721525_26c601ed62_b.jpg
Downloaded: https://live.staticflickr.com/3721/11041116835_97f168e141_b.jpg
Downloaded: https://live.staticflickr.com/1417/770483110_a69478d1f7_b.jpg
Downloaded: https://live.staticflickr.com/7119/6893979886_4a3256cef8_b.jpg
Downloaded: https://live.staticflickr.com/5147/5672446288_37d3b8d7b8_b.jpg
Downloaded: https://live.staticflickr.com/8594/27983395304_e3fba82d5b_b.jpg
Downloaded: https://l

# Data Preprocessing

In [24]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [25]:
def check_and_preprocess_images(image_dir):
  for root, _, files in os.walk(image_dir):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    # Check if image is smaller than 50x50 pixels
                    if img.size[0] < 50 or img.size[1] < 50:
                        os.remove(file_path)
                        print(f"Deleted {file_path}: Image too small ({img.size[0]}x{img.size[1]})")
                        continue

                    # Convert non-RGB images to RGB
                    if img.mode != 'RGB':
                        img = img.convert('RGB')
                        img.save(file_path)
                        print(f"Converted {file_path} to RGB")

            except Exception as e:
                # If file is not an image, delete it
                os.remove(file_path)
                print(f"Deleted {file_path}: Not an image or corrupted file ({str(e)})")

# Example usage:
check_and_preprocess_images('Dataset')

Converted Dataset/scenery/desert/48149465666_a6598a3d03_b.jpg to RGB
Converted Dataset/scenery/desert/52712003999_cba6904a86_b.jpg to RGB
Converted Dataset/plant/Bean/8529951413_c315df19e5_b.jpg to RGB
Converted Dataset/furniture/spoon/16038961387_c06564b0d8_b.jpg to RGB
Converted Dataset/furniture/table/14443559552_89c59a16a4_b.jpg to RGB


## Folder Construction

In [26]:
!gdown --id 1--6fe48D9ydnTpLV1GKKqJ0pqpOXB3z_

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1--6fe48D9ydnTpLV1GKKqJ0pqpOXB3z_
From (redirected): https://drive.google.com/uc?id=1--6fe48D9ydnTpLV1GKKqJ0pqpOXB3z_&confirm=t&uuid=22fe8c9c-d3ff-48a3-9e81-7bb58bb1fa9b
To: /content/Clean_Dataset.zip
100% 483M/483M [00:05<00:00, 89.2MB/s]


In [27]:
!unzip Clean_Dataset

Archive:  Clean_Dataset.zip
replace Dataset/plant/Clove/53742600482_be65dab11c_b.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Dataset/plant/Clove/53742600482_be65dab11c_b.jpg  
replace Dataset/plant/Clove/6499672315_313ec22f51_b.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Dataset/plant/Clove/6499672315_313ec22f51_b.jpg  
replace Dataset/plant/Clove/22440611562_e293ed2cb6_b.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [28]:
import os
import shutil
from collections import defaultdict

# Define the source and target directories
source_dir = "Dataset"
train_dir = "data/train"
test_dir = "data/test"

# Create the target directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Initialize a dictionary to hold file paths for each class
class_files = defaultdict(list)

# Read the file paths from the text file
with open('filename.txt', 'r') as file:
    lines = file.readlines()
    for line in lines:
        line = line.strip()
        if line:
            # Extract the class name from the path
            parts = line.split('/')
            class_name = parts[2]  # Structure Dataset/category/class/image.jpg
            class_files[class_name].append(line)

# Move images to the train and test directories
for class_name, files in class_files.items():
    # Create the train and test directories for the class
    train_class_dir = os.path.join(train_dir, class_name)
    test_class_dir = os.path.join(test_dir, class_name)
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)

    # Move 19 images to train and 1 image to test
    for i, file_path in enumerate(files):
        if i == 0:
            shutil.copy(file_path, test_class_dir)
        elif i < 20:
            shutil.copy(file_path, train_class_dir)

print("Dataset organization complete!")

Dataset organization complete!


In [29]:
!zip -r /content/drive/MyDrive/data.zip data

  adding: data/ (stored 0%)
  adding: data/train/ (stored 0%)
  adding: data/train/cave/ (stored 0%)
  adding: data/train/cave/6166393867_a8af73f7e8_b.jpg (deflated 0%)
  adding: data/train/cave/3498867273_1f1531cb43_b.jpg (deflated 0%)
  adding: data/train/cave/8591351967_f641b5e54d_b.jpg (deflated 0%)
  adding: data/train/cave/6026311122_b944cddbff_b.jpg (deflated 0%)
  adding: data/train/cave/15511172723_4cdb51fd43_b.jpg (deflated 1%)
  adding: data/train/cave/6276863304_4b9f30551f_b.jpg (deflated 0%)
  adding: data/train/cave/3702454576_7d48d99ae0_b.jpg (deflated 0%)
  adding: data/train/cave/3648646409_4f0eb0e78b_b.jpg (deflated 0%)
  adding: data/train/cave/3351509570_c50a2ec631_b.jpg (deflated 0%)
  adding: data/train/cave/3617208393_ab78f84357_b.jpg (deflated 0%)
  adding: data/train/cave/35942145413_471121736a_b.jpg (deflated 0%)
  adding: data/train/cave/36613154921_367af96f82_b.jpg (deflated 0%)
  adding: data/train/cave/36743755995_2ed9609550_b.jpg (deflated 0%)
  adding: d